In [78]:
!pip install datasets sentencepiece

In [79]:
import pandas as pd
from transformers import AutoTokenizer


In [80]:
!pip install amseg

  Preparing metadata (setup.py) ... done
  Created wheel for amseg: filename=amseg-2.3-py3-none-any.whl size=9773 sha256=ffe84d06995295e849d15dc3996f8f6e46d0970c8a0ee4978b39a07ec95aedf9
  Stored in directory: /root/.cache/pip/wheels/df/9c/55/1368cec635d34f48f9ad4c7ae04d120b7acea9e371f3974d6f
Successfully built amseg


In [81]:
!pip install pdf2image

In [82]:
!pip install pytesseract

In [83]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [84]:
with open('/content/drive/My Drive/KAIM 3/WEEK-5/labeled_telegram_product_price_location.txt', 'r') as file:
    lines = file.readlines()

# Process lines as needed
data = [line.strip().split('\t') for line in lines]  # Adjust the split based on your delimiter
df = pd.DataFrame(data)

In [85]:
df

,0
0,እንኳን O
1,ለጥምቀት O
2,በዓል O
3,በሰላም O
4,አደረሳችሁ! O
...,...
145848,ተጭነው O
145849,ያድርጉ፣ O
145850,ቤተሰብ O
145851,ይሁኑ O


In [86]:
# Process lines: separate tokens and labels
data = [line.strip().split() for line in lines if line.strip()]  # Split based on spaces
tokens = [item[0] for item in data]  # Extract tokens
labels = [item[1] for item in data]  # Extract labels

In [89]:
data

[['እንኳን', 'O'],
 ['ለጥምቀት', 'O'],
 ['በዓል', 'O'],
 ['በሰላም', 'O'],
 ['አደረሳችሁ!', 'O'],
 [',', 'O'],
 ['መልካም', 'O'],
 ['በዓል', 'O'],
 ["'", 'O'],
 ['የፀጉር', 'B-PRODUCT'],
 ['መቁረጫ', 'B-PRODUCT'],
 ['4', 'O'],
 ['240', 'O'],
 ['5000', 'O'],
 ['4', 'O'],
 ['(3,6,10,13', 'O'],
 [')', 'O'],
 ['ዋጋ፦', 'B-PRICE'],
 ['2600', 'I-PRICE'],
 ['ብር', 'I-PRICE'],
 ['ከነፃ', 'O'],
 ['ማድረስ', 'O'],
 ['ጋር', 'O'],
 ['ውስን', 'O'],
 ['ፍሬ', 'O'],
 ['ነው', 'O'],
 ['የቀረው', 'O'],
 ['<', 'O'],
 ['-', 'O'],
 ['./', 'O'],
 ['ጥራት', 'O'],
 ['ዋስትና', 'O'],
 ['ቅናሽ', 'O'],
 ['አድራሻ', 'O'],
 ['፦', 'O'],
 ['ቁጥር', 'O'],
 ['1', 'O'],
 [':-', 'O'],
 ['መገናኛ', 'B-LOC'],
 ['ዘፍመሽ', 'I-LOC'],
 ['ግራንድ', 'I-LOC'],
 ['ሞል', 'I-LOC'],
 ['3', 'O'],
 ['ተኛ', 'O'],
 ['ፎቅ', 'O'],
 ['ቁጥር', 'O'],
 ['329', 'O'],
 ['ቁጥር', 'O'],
 ['2', 'O'],
 [':-', 'O'],
 ['ጀሞ', 'B-LOC'],
 ['1', 'I-LOC'],
 ['ከለላ', 'I-LOC'],
 ['ህንፃ', 'I-LOC'],
 ['ግራውንድ', 'O'],
 ['ለይ', 'O'],
 ['07', 'O'],
 ['ማሳሰብያ:', 'O'],
 ['ሱቃችን', 'O'],
 ['ሲመጡ', 'O'],
 ['ትክክለኛ', 'O'],
 ['የኛ', 'O'],
 ['ሱቅ',

In [88]:
# Initialize the tokenizer
model_name = "xlm-roberta-base"  # Change to any approprate model from Hugging Face if needed
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [90]:
# Function to tokenize and align labels
def tokenize_and_align_labels(tokenizer, tokens, labels):
    aligned_tokens = []
    aligned_labels = []

    # Tokenize each word/token with associated label
    for word, label in zip(tokens, labels):
        tokenized_word = tokenizer.tokenize(word)  # Tokenize the word
        aligned_tokens.extend(tokenized_word)  # Add tokens to the list

        # Assign the label to the first subtoken and 'O' to subsequent subtokens
        aligned_labels.extend([label] + ['O'] * (len(tokenized_word) - 1))

    return aligned_tokens, aligned_labels

In [91]:
# Tokenize and align labels
aligned_tokens, aligned_labels = tokenize_and_align_labels(tokenizer, tokens, labels)

In [92]:
# Display the first 10 results
for token, label in zip(aligned_tokens[:20], aligned_labels[:20]):
    print(f"{token:20} {label}")


▁እንኳን                O
▁ለ                   O
ጥም                   O
ቀት                   O
▁በዓል                 O
▁በሰላም                O
▁አ                   O
ደረ                   O
ሳ                    O
ችሁ                   O
!                    O
▁                    O
,                    O
▁መልካም                O
▁በዓል                 O
▁'                   O
▁የ                   B-PRODUCT
ፀ                    O
ጉ                    O
ር                    O


In [93]:
from amseg.amharicSegmenter import AmharicSegmenter


In [94]:
# Initialize the Amharic segmenter
sent_punct = []
word_punct = []
segmenter = AmharicSegmenter(sent_punct, word_punct)

In [95]:
# Process lines: separate tokens and labels
data = [line.strip().split() for line in lines if line.strip()]  # Split based on spaces
tokens = [item[0] for item in data]  # Extract tokens
labels = [item[1] for item in data]  # Extract labels

In [96]:
# Function to align tokens with their respective labels
def align_tokens_with_labels(tokenizer, tokens, labels):
    aligned_tokens = []
    aligned_labels = []

    # Tokenize each word/token with associated label
    for word, label in zip(tokens, labels):
        tokenized_word = tokenizer.amharic_tokenizer(word)  # Tokenize the word
        aligned_tokens.extend(tokenized_word)  # Add tokens to the list

        # Assign the label to the first subtoken and 'O' to subsequent subtokens
        aligned_labels.extend([label] + ['O'] * (len(tokenized_word) - 1))

    return aligned_tokens, aligned_labels

In [97]:
# Example usage with different variable names
new_tokens, new_labels = align_tokens_with_labels(segmenter, tokens, labels)

In [98]:
# Output the first 20 aligned tokens and labels
for token, label in zip(new_tokens[:20], new_labels[:20]):
    print(f"{token}: {label}")


እንኳን: O
ለጥምቀት: O
በዓል: O
በሰላም: O
አደረሳችሁ: O
!: O
,: O
መልካም: O
በዓል: O
': O
የፀጉር: B-PRODUCT
መቁረጫ: B-PRODUCT
4: O
240: O
5000: O
4: O
(3,6,10,13: O
): O
ዋጋ: B-PRICE
፦: O


In [99]:
# Saving the final tokens and labels to a CSV file
output_df = pd.DataFrame({'Token': new_tokens, 'Label': new_labels})
output_df.to_csv('/content/drive/My Drive/KAIM 3/WEEK-5/final_tokens_labels.csv', index=False)

In [100]:
from datasets import load_dataset
import sentencepiece as spm

In [101]:
#Load the dataset
dataset = load_dataset("israel/Amharic-News-Text-classification-Dataset")

README.md:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/150M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/37.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/41186 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10297 [00:00<?, ? examples/s]

In [102]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'headline', 'category', 'date', 'views', 'article', 'link', 'word_len'],
        num_rows: 41186
    })
    test: Dataset({
        features: ['Unnamed: 0', 'headline', 'category', 'date', 'views', 'article', 'link', 'word_len'],
        num_rows: 10297
    })
})

In [103]:
# Extract the text from the dataset
texts = dataset['train']['article']

In [104]:
texts

['The Ethiopian national side continued on their unimpressive steak of results when they were trashed 40 against home based Moroccos national team in a friendly game played out in Rabat The game was originally slated for Tuesday but the teams were forced to play on Monday evening without any explanation on why the encounter rescheduled The Royal Moroccan Football Federation FRMF invited Ethiopia to play a friendly game in Rabat last week The Waliyas were beaten 20 to Botswana a week earlier\nThe home side took a three goals cushion in the first interval before extending their lead to 40 following the resumption Ethiopian center forward Getaneh Kebede had received his marching order after getting a second booking The home based Moroccan side used the game as part of their preps for the upcoming Total African Nations Championship CHANWant away Ethiopia coach Ashenafi Bekele has deployed some of the national side played out of their position during the duel The traveling squad of Walias c

In [105]:
# Filter out None values and prepare the data for SentencePiece
filtered_texts = [text for text in texts if text is not None]  # Remove None values
text_data = "\n".join(filtered_texts)

In [106]:
# Save to a temporary file
with open('temp_text.txt', 'w', encoding='utf-8') as f:
    f.write(text_data)

In [107]:
# Train SentencePiece model
spm.SentencePieceTrainer.train('--input=temp_text.txt --model_prefix=AmharicSPM --vocab_size=100000')

In [109]:
tokenizer = spm.SentencePieceProcessor(model_file='AmharicSPM.model')

In [110]:
# Tokenize the input text
text = "ቁ.2 ለቡ  መዳህኒዓለም ቤተ/ክርስቲያን ፊት ለፊት  #ዛም_ሞል 2ኛ ፎቅ ቢሮ ቁጥር.214"
tokens = tokenizer.encode(text, out_type=str)  # Use encode method for tokenization
#Print the tokenized words
print(tokens)

['▁ቁ', '.', '2', '▁ለቡ', '▁መዳ', 'ህ', 'ኒ', 'ዓ', 'ለም', '▁ቤተ', '/', 'ክርስቲያን', '▁ፊት', '▁ለፊት', '▁', '#', 'ዛ', 'ም', '_', 'ሞል', '▁2', 'ኛ', '▁ፎቅ', '▁ቢሮ', '▁ቁጥር', '.', '214']


In [111]:
# Process lines: separate tokens and labels
data = [line.strip().split() for line in lines if line.strip()]  # Split based on spaces
tokens = [item[0] for item in data]  # Extract tokens
labels = [item[1] for item in data]  # Extract labels

In [112]:
tokenizer = spm.SentencePieceProcessor(model_file='AmharicSPM.model')

In [113]:
def tokenize_and_align_labels(tokenizer, tokens, labels):
    aligned_tokens = []
    aligned_labels = []
    token_ids = []

    for word, label in zip(tokens, labels):
        tokenized_ids = tokenizer.encode(word, out_type=int)  # Get token IDs
        tokenized_words = tokenizer.decode(tokenized_ids).split()  # Decode back to words

        aligned_tokens.extend(tokenized_words)  # Add decoded words to the list
        token_ids.extend(tokenized_ids)  # Add token IDs to the list

        aligned_labels.extend([label] + ['O'] * (len(tokenized_words) - 1))

    return aligned_tokens, token_ids, aligned_labels


In [114]:
# Tokenize and align labels
custom_tokens, custom_ids, custom_labels = tokenize_and_align_labels(tokenizer, tokens, labels)


In [115]:
# Display results from index 50 to 80 with tokenized words, their IDs, and labels
for token, token_id, label in zip(custom_tokens[:20], custom_ids[:20], custom_labels[:20]):
    print(f"{token:<25} {token_id:<10} {label}")


እንኳን                      632        O
ለጥምቀት                     24422      O
በ                         7          O
⁇                         0          O
ል                         124        O
በሰላም                      1031       O
አደረሳችሁ                    5003       O
⁇                         0          O
⁇                         8          O
መልካም                      0          O
በ                         405        O
⁇                         7          O
ል                         0          O
⁇                         124        O
የፀጉር                      8          B-PRODUCT
መቁረጫ                      0          B-PRODUCT
4                         12         O
240                       22154      O
5000                      47687      O
4                         269        O


In [123]:
# Step 6: Create the Final Dataset
final_data_dict = {'tokens': [aligned_tokens], 'labels': [aligned_labels]}
final_dataset = Dataset.from_dict(final_data_dict)


In [124]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",  # Save the model at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

In [125]:
# Step 8: Load Pre-Trained Model and Initialize Trainer
from transformers import AutoModelForTokenClassification
from seqeval.metrics import f1_score, precision_score, recall_score

In [126]:
# Load the model for token classification (NER)
model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=7)


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [127]:
# Define the compute_metrics function
def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)  # Get predicted labels
    true_labels = labels.numpy()  # Convert labels to numpy array

    # Convert predictions and labels to a list of tokens for seqeval
    pred_tokens = [tokenizer.decode(pred) for pred in predictions]
    true_tokens = [tokenizer.decode(label) for label in true_labels]

    # Evaluate precision, recall, and F1 score using seqeval
    precision = precision_score(true_tokens, pred_tokens)
    recall = recall_score(true_tokens, pred_tokens)
    f1 = f1_score(true_tokens, pred_tokens)

    return {"precision": precision, "recall": recall, "f1": f1}

In [128]:
 # Step 9: Initialize Trainer and Fine-Tune the Model
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=final_dataset,  # Preprocessed training dataset
    eval_dataset=final_dataset,   # Preprocessed validation dataset
    compute_metrics=compute_metrics
)

In [130]:
# Start fine-tuning
trainer.train()

ValueError: too many dimensions 'str'

In [ ]:
# Step 10: Evaluate the Model and Save It
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained('./final_ner_model')
tokenizer.save_pretrained('./final_ner_model')

print("Model saved in './final_ner_model'")